导入pyspark 并创建sparkSession

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('work').getOrCreate()

读取数据文件 `walmart_stock.csv`

In [2]:
df = spark.read.csv("walmart_stock.csv", inferSchema=True, header=True)

数据有哪些列？

In [5]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

数据的 schema 是什么？

In [7]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



打印数据的前5行

In [8]:
df.head(5)

[Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

打印数字, 字符列的统计信息 

In [9]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

查看统计表的schema

In [10]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



把统计表的字段转换成 float类型

In [21]:
from pyspark.sql.functions import format_number
res = df.describe()
res.select(res['summary'],
           format_number(res.Open.cast('float'),2).alias('Open'),
           format_number(res.High.cast('float'),2).alias('High'),
           format_number(res.Low.cast('float'),2).alias('Low'),
           format_number(res.Close.cast('float'),2).alias('Close'),
           res['Volume'].cast('int').alias('Volume')
           ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



In [26]:
df2 = df.withColumn("HD Ratio", df['High']/df['Low'])
df2.select(['Date','HD Ratio']).show()

+-------------------+------------------+
|               Date|          HD Ratio|
+-------------------+------------------+
|2012-01-03 00:00:00|1.0198764326019114|
|2012-01-04 00:00:00|1.0147973261342302|
|2012-01-05 00:00:00|1.0214151108688558|
|2012-01-06 00:00:00|1.0098522508892858|
|2012-01-09 00:00:00| 1.010692481693567|
|2012-01-10 00:00:00|1.0123770600203459|
|2012-01-11 00:00:00|1.0082994239786682|
|2012-01-12 00:00:00|1.0101009760908761|
|2012-01-13 00:00:00|  1.01016781935834|
|2012-01-17 00:00:00|1.0099126512096774|
|2012-01-18 00:00:00|1.0063704440445784|
|2012-01-19 00:00:00|1.0164016736401673|
|2012-01-20 00:00:00| 1.009559947570791|
|2012-01-23 00:00:00|1.0077673444973507|
|2012-01-24 00:00:00|1.0205761316872428|
|2012-01-25 00:00:00| 1.009338138772311|
|2012-01-26 00:00:00|1.0176073720585816|
|2012-01-27 00:00:00|1.0095804094882654|
|2012-01-30 00:00:00| 1.016072941709128|
|2012-01-31 00:00:00| 1.016341993517927|
+-------------------+------------------+
only showing top

获取close列的平均值

In [30]:
from pyspark.sql.functions import mean
df.select(mean(df['Close'])).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



Volume 列的最大值和最小值是是什么？

In [31]:
from pyspark.sql.functions import max, min
df.select(max('volume'), min('volume')).show()

+-----------+-----------+
|max(volume)|min(volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



筛选出来 close 列小于 60 的列

In [34]:
df.filter(df['Close'] < 60).count()

81